# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh our knowledge of SQL Queries!

## Objectives

You will be able to:
- Practice your SQL knowledge


## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as `pandas`. By combining them, we'll be able to write our queries as python strings, and make sure that the results are always returned as a pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3`
* Establish a connection to the database `data.sqlite`
* Get the `cursor` from the connection and store it in the variable `c`.

In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

* Write a query that gets the first name, last name, phone number, and address for all customers in California with a credit limit greater than 25000.00. 

In [4]:
# For the first query, the boilerplate for getting 
#the query into a dataframe has been provided for you
c.execute("""SELECT contactFirstName, contactLastName, phone, addressline1, creditLimit from customers
             WHERE creditLimit > 25000.00 and state = 'CA'
             LIMIT 5
             """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Susan,Nelson,4155551450,5677 Strong St.,210500.00
1,Julie,Murphy,6505555787,5557 North Pendale Street,64600.00
2,Juri,Hashimoto,6505556809,9408 Furth Circle,84600.00
3,Julie,Young,6265557265,78934 Hillside Dr.,90700.00
4,Mary,Young,3105552373,4097 Douglas Av.,11000.00


In [5]:
## does not work -- don't use strings with numerical operators!

# c.execute("""SELECT contactFirstName, contactLastName, phone, addressline1, creditLimit from customers
#              WHERE creditLimit > '25000' and state = 'CA' order by creditLimit desc
#              """)
# df = pd.DataFrame(c.fetchall())
# df.columns = [x[0] for x in c.description]
# df

In [6]:
c.execute("""SELECT contactFirstName, contactLastName, phone, addressline1, creditLimit from customers
             WHERE CAST (creditLimit AS DECIMAL (6,2)) > 25000.00 and state = 'CA' 
             ORDER BY CAST (creditLimit AS DECIMAL (6,2)) DESC
             """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Susan,Nelson,4155551450,5677 Strong St.,210500.00
1,Valarie,Thompson,7605558146,361 Furth Circle,105000.00
2,Julie,Brown,6505551386,7734 Strong St.,105000.00
3,Julie,Young,6265557265,78934 Hillside Dr.,90700.00
4,Juri,Hashimoto,6505556809,9408 Furth Circle,84600.00
5,Sue,Frick,4085553659,3086 Ingle Ln.,77600.00
6,Julie,Murphy,6505555787,5557 North Pendale Street,64600.00
7,Sue,Taylor,4155554312,2793 Furth Circle,60300.00
8,Brian,Chandler,2155554369,6047 Douglas Av.,57700.00
9,Steve,Thompson,3105553722,3675 Furth Circle,55400.00


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that get sthe average credit limit per state.

In [7]:
c.execute("""SELECT state, AVG (creditLimit) from customers
             GROUP BY state
             ORDER BY AVG (creditLimit) DESC
             LIMIT 5
             """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,state,AVG (creditLimit)
0,NH,114200.000000
1,NSW,100550.000000
2,Tokyo,94400.000000
3,Isle of Wight,93900.000000
4,NY,89966.666667


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [8]:
c.execute("""SELECT c.customerName, c.customerNumber, o.orderNumber, o.status, od.quantityOrdered from customers c
            JOIN orders o on
            c.customerNumber = o.customerNumber
            JOIN orderdetails od on 
            o.orderNumber = od.orderNumber
            LIMIT 5
             """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Atelier graphique,103,10123,Shipped,26
1,Atelier graphique,103,10123,Shipped,34
2,Atelier graphique,103,10123,Shipped,46
3,Atelier graphique,103,10123,Shipped,50
4,Atelier graphique,103,10298,Shipped,32


#### Expected Output

<img src='images/expected-output-3.png'>

## HAVING and ORDER BY

Now, repeat the last query, but only get orders from customers that have a quantityOrdered value greater than 30. Sort the rows in ascending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY--make sure you get the order of them correct!

In [9]:
c.execute("""SELECT c.customerName, c.customerNumber, o.orderNumber, o.status, od.quantityOrdered 
            FROM customers c
            JOIN orders o on
            c.customerNumber = o.customerNumber
            JOIN orderdetails od on 
            o.orderNumber = od.orderNumber
            GROUP BY c.customerNumber
            HAVING SUM (od.quantityOrdered)>30
            ORDER BY od.quantityOrdered ASC
            LIMIT 5
             """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,"Tokyo Collectables, Ltd",398,10408,Shipped,15
1,Petit Auto,314,10423,In Process,31
2,Frau da Collezione,473,10218,Shipped,34
3,"Clover Collections, Co.",189,10297,Shipped,35
4,Boards & Toys Co.,219,10376,Shipped,35


In [10]:
# c.execute("""SELECT c.customerName, c.customerNumber, o.orderNumber, o.status, od.quantityOrdered FROM Customers c JOIN Orders o 
# ON c.customerNumber = o.customerNumber JOIN OrderDetails od ON od.orderNumber = o.orderNumber
# GROUP BY od.quantityOrdered
# HAVING SUM(od.quantityOrdered) > 10 
# ORDER BY od.quantityOrdered ASC""")
# df = pd.DataFrame(c.fetchall())
# df.columns = [x[0] for x in c.description]
# df.head()

#### Expected Output

<img src='images/expected-output-4.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from an office with less than 5 employees. 

In [45]:
c.execute('''select * from offices''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan


In [46]:
c.execute("""SELECT officeCode FROM offices o
    JOIN employees e
    using (officecode)""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()


,officeCode
0,1
1,1
2,1
3,1
4,1


In [48]:
c.execute("""SELECT o.officeCode FROM offices o
    JOIN employees e
    on o.officecode = e.officecode
    GROUP BY o.officeCode""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df


,officeCode
0,1
1,2
2,3
3,4
4,5
5,6
6,7


In [42]:
c.execute("""SELECT o.officeCode FROM offices o
    JOIN employees e
   on o.officecode = e.officecode
    GROUP BY o.officeCode
    HAVING COUNT(e.employeeNumber) < 5"""
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()



,officeCode
0,2
1,3
2,5
3,6
4,7


In [21]:
c.execute("""select lastName, firstName, employeeNumber, officeCode
                    FROM employees
                    WHERE officeCode IN (SELECT officeCode 
                                                FROM offices 
                                                JOIN employees
                                                USING(officeCode)
                                                GROUP BY officeCode
                                                HAVING COUNT(employeeNumber) < 5
                                         );
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,lastName,firstName,employeeNumber,officeCode
0,Patterson,William,1088,6
1,Firrelli,Julie,1188,2
2,Patterson,Steve,1216,2
3,Tseng,Foon Yue,1286,3
4,Vanauf,George,1323,3


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!